# Metadata

**L1 Taxonomy** - Security

**L2 Taxonomy** - Application Security

**Subtopic** - Applying secure headers in a Flask app using extensions like Flask-Talisman

**Use Case** - Develop a simple Python script that uses Flask-Talisman to apply secure headers to a Flask application. The script should demonstrate how to set up Flask-Talisman, configure secure headers such as Content Security Policy (CSP), HTTP Strict Transport Security (HSTS), and X-Content-Type-Options, and apply these headers to a basic Flask route.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
flask==2.3.3
flask-talisman==1.1.0
```


# Prompt

## Problem Overview

You are to implement an advanced Python module that enforces secure HTTP headers on a Flask web application, integrating **per-route configuration** and **dynamic Content Security Policy (CSP) nonces**. This security module must allow specifying custom security rules for individual routes or blueprints and ensure that every HTTP response adheres to the assigned policy. CSP nonces must be cryptographically secure and injected dynamically into both HTTP headers and rendered HTML templates to support secure inline scripts and styles.

Your solution is intended for use in web systems that handle sensitive data or interactive UI components and must be both flexible and robust in the face of real-world demands.

## Input Format

- A Flask `app` object representing the target application.
- An optional default configuration dictionary specifying:
    - `'csp'`: Default CSP directives (dictionary).
    - `'hsts'`: Default HSTS max-age (integer, seconds).
    - `'x_content_type_options'`: Apply X-Content-Type-Options (bool).
- An optional **route_config** dictionary mapping route patterns or endpoint names to per-route configuration dictionaries, following the same structure as the default config.
- Routes may be registered before or after applying the security module.
- Each HTML template rendered for a route must have the ability to receive the generated CSP nonce for use in inline scripts or styles.

## Output Format

- No direct return value.
- The Flask app responds as normal, but each route's HTTP response includes headers in accordance with its configuration-global defaults or per-route overrides.
- Inline scripts or styles in HTML responses can safely use the automatically injected nonce.

## Examples

**Given**:

```python
default_config = {
    'csp': {'default-src': ["'self'"]},
    'hsts': 86400,
    'x_content_type_options': True
}
route_config = {
    '/public': {
        'csp': {'default-src': ["'self'", 'cdnjs.cloudflare.com']},
        'hsts': 0
    },
    '/dashboard': {
        'csp': {'default-src': ["'self'"], 'script-src': ["'self'", "'nonce-{nonce}'"]},
        'hsts': 31536000
    }
}
from advanced_secure_headers import apply_advanced_secure_headers, get_current_nonce
app = Flask(__name__)

apply_advanced_secure_headers(app, default_config, route_config)

@app.route("/dashboard")
def dashboard():
    nonce = get_current_nonce()
    return render_template("dashboard.html", nonce=nonce)
```

**A GET request to `/dashboard` might return**:

- **Headers:**
    - `Content-Security-Policy: default-src 'self'; script-src 'self' 'nonce-'`
    - `Strict-Transport-Security: max-age=31536000`
    - `X-Content-Type-Options: nosniff`
- **Response HTML includes:**  
    `/* safe inline JS */`


# Requirements

## Explicit Requirements

- Support per-route or per-endpoint custom configuration, overriding defaults as needed.
- Dynamically generate a secure, unique nonce for each request needing it.
- Nonce is automatically injected into headers and made available for templates in that request's context.
- Must validate input configs and raise clear errors for invalid or conflicting settings.
- All header logic (global defaults and per-route settings) must reside in an importable, isolated module.
- No global state; all state must be strictly per-request or tied to the app instance.
- Configurable CSP, HSTS, and X-Content-Type-Options as described.
- Must fall back to defaults if no route-specific configuration is given for a request.
- For non-HTML responses, headers must still apply, but nonce injection is not required.

## Implicit Requirements

- Use Flask-Talisman and allow per-route adjustments in the solution.
- All route registration and security application must function regardless of registration order.
- The module's API must provide a helper function to retrieve the current request's CSP nonce for template use.
- Prevent duplication or collision of after_request or template context handlers.
- Code must be thoroughly documented and structured for clarity and easy integration.

## Function Signature(s) and/or Class Definition(s)

Implement at least the following interface:

```python
def apply_advanced_secure_headers(app: 'Flask', default_config: dict = None, route_config: dict = None) -> None:
    """
    Applies secure HTTP headers to the Flask app, with dynamic per-route control and nonce management.
    """

def get_current_nonce() -> str:
    """
    Returns the current request's CSP nonce, or None if not applicable.
    """
```

You may include additional classes, decorators, or helper functions as needed.

## Edge Cases

- Overlapping routes/endpoints with different configs.
- Routes registered after security headers are applied.
- Config key omissions or misconfigurations.
- Binary and non-HTML content (nonce not injected, but headers still must be correct).
- Nested blueprints or complex routing structures.
- Concurrent requests receiving different nonces.
- Route that disables CSP or HSTS entirely (should be possible).

## Constraints

- Must be compatible with Flask 2.x and Flask-Talisman 1.x or later.
- Nonce values must be cryptographically secure and random for each request.
- No use of global or module-level state for nonce or header management.
- All HTTP headers must comply with current security best practices and standards.
- Module should not interfere with unrelated app extensions or middleware.

In [ ]:
# code

"""
Advanced Flask Security Headers Module

This module provides comprehensive security header management for Flask
applications with per-route configuration and dynamic CSP nonce support.
"""

import secrets
import copy
from typing import Dict, Any, Optional, List
from flask import Flask, g, request
from flask_talisman import Talisman


def apply_advanced_secure_headers(
    app: Flask,
    default_config: Optional[Dict[str, Any]] = None,
    route_config: Optional[Dict[str, Dict[str, Any]]] = None
) -> None:
    """
    Apply secure HTTP headers to Flask app with per-route configuration.

    Args:
        app: Flask application instance to secure
        default_config: Global security configuration dictionary with keys:
            - 'csp': Dict of CSP directives (e.g., {'default-src': ["'self'"]})
            - 'hsts': Integer for HSTS max-age in seconds (>= 0)
            - 'x_content_type_options': Boolean for X-Content-Type-Options
        route_config: Dict mapping route paths/endpoints to per-route configs
            with same structure as default_config

    Raises:
        ValueError: If configuration contains invalid values
        TypeError: If configuration contains invalid types

    Example:
        >>> app = Flask(__name__)
        >>> default_cfg = {
        ...     'csp': {'default-src': ["'self'"]},
        ...     'hsts': 86400,
        ...     'x_content_type_options': True
        ... }
        >>> route_cfg = {
        ...     '/api': {'hsts': 0},
        ...     '/dashboard': {
        ...         'csp': {'script-src': ["'self'", "'nonce-{nonce}'"]}
        ...     }
        ... }
        >>> apply_advanced_secure_headers(app, default_cfg, route_cfg)
    """
    if default_config is None:
        default_config = {}
    if route_config is None:
        route_config = {}

    # Validate configurations early
    _validate_config("default_config", default_config)
    for route_key, config in route_config.items():
        _validate_config(f"route_config['{route_key}']", config)

    # Store validated configs in app extensions
    if not hasattr(app, 'extensions'):
        app.extensions = {}
    app.extensions['advanced_secure_headers'] = {
        'default_config': default_config,
        'route_config': route_config
    }

    # Initialize base Talisman with minimal configuration
    # We'll override headers in our custom handlers
    Talisman(
        app,
        force_https=False,  # We'll handle this per-route
        strict_transport_security=False,  # We'll handle this per-route
        content_security_policy=False,  # We'll handle this per-route
        content_security_policy_nonce_in=[],  # We manage nonces ourselves
    )

    @app.before_request
    def _generate_nonce_for_request() -> None:
        """
        Generate cryptographically secure nonce for each request.

        The nonce is stored in Flask's g object and made available for
        template context and CSP header generation.
        """
        # Check if current route needs a nonce
        merged_config = _get_merged_config_for_request(app)
        csp_config = merged_config.get('csp', {})

        if _csp_requires_nonce(csp_config):
            g.csp_nonce = secrets.token_urlsafe(16)

    @app.after_request
    def _apply_security_headers(response):
        """
        Apply security headers based on merged route configuration.

        Args:
            response: Flask response object

        Returns:
            Modified response with appropriate security headers
        """
        merged_config = _get_merged_config_for_request(app)

        # Apply HSTS
        hsts_max_age = merged_config.get('hsts', 0)
        if hsts_max_age > 0:
            response.headers['Strict-Transport-Security'] = (
                f'max-age={hsts_max_age}'
            )
        else:
            # Remove HSTS header if disabled for this route
            response.headers.pop('Strict-Transport-Security', None)

        # Apply X-Content-Type-Options
        if merged_config.get('x_content_type_options', False):
            response.headers['X-Content-Type-Options'] = 'nosniff'
        else:
            response.headers.pop('X-Content-Type-Options', None)

        # Apply CSP
        csp_config = merged_config.get('csp')
        if csp_config is not None:
            csp_header = _build_csp_header(csp_config, get_current_nonce())
            if csp_header:
                response.headers['Content-Security-Policy'] = csp_header
        else:
            # CSP explicitly disabled for this route
            response.headers.pop('Content-Security-Policy', None)

        return response

    @app.context_processor
    def _inject_nonce_context() -> Dict[str, Any]:
        """
        Inject nonce into template context for easy access.

        Returns:
            Dictionary containing nonce value for template use
        """
        return {'nonce': get_current_nonce()}


def get_current_nonce() -> Optional[str]:
    """
    Return the current request's CSP nonce.

    Returns:
        Cryptographically secure nonce string or None if not generated

    Example:
        >>> # In a view function or template
        >>> nonce = get_current_nonce()
        >>> # nonce might be 'abc123def456' or None
    """
    return getattr(g, 'csp_nonce', None)


def _validate_config(config_name: str, config: Dict[str, Any]) -> None:
    """
    Validate security configuration dictionary.

    Args:
        config_name: Name of config for error messages
        config: Configuration dictionary to validate

    Raises:
        TypeError: If config values have wrong types
        ValueError: If config values are invalid
    """
    if not isinstance(config, dict):
        raise TypeError(f"{config_name} must be a dictionary")

    # Validate HSTS
    if 'hsts' in config:
        hsts_value = config['hsts']
        if not isinstance(hsts_value, int):
            raise TypeError(
                f"{config_name}['hsts'] must be an integer (seconds)"
            )
        if hsts_value < 0:
            raise ValueError(
                f"{config_name}['hsts'] must be non-negative"
            )

    # Validate X-Content-Type-Options
    if 'x_content_type_options' in config:
        xcto_value = config['x_content_type_options']
        if not isinstance(xcto_value, bool):
            raise TypeError(
                f"{config_name}['x_content_type_options'] must be a boolean"
            )

    # Validate CSP
    if 'csp' in config:
        csp_value = config['csp']
        if csp_value is not None and not isinstance(csp_value, dict):
            raise TypeError(
                f"{config_name}['csp'] must be a dictionary or None"
            )
        if isinstance(csp_value, dict):
            _validate_csp_config(config_name, csp_value)

    # Check for unknown keys
    valid_keys = {'csp', 'hsts', 'x_content_type_options'}
    unknown_keys = set(config.keys()) - valid_keys
    if unknown_keys:
        raise ValueError(
            f"{config_name} contains unknown keys: {unknown_keys}"
        )


def _validate_csp_config(config_name: str, csp_config: Dict[str, Any]) -> None:
    """
    Validate CSP configuration dictionary.

    Args:
        config_name: Name of config for error messages
        csp_config: CSP configuration dictionary

    Raises:
        TypeError: If CSP directive values have wrong types
        ValueError: If CSP directive values are invalid
    """
    for directive, sources in csp_config.items():
        if not isinstance(directive, str):
            raise TypeError(
                f"{config_name}['csp'] directive keys must be strings"
            )

        if not isinstance(sources, (list, tuple)):
            raise TypeError(
                f"{config_name}['csp']['{directive}'] must be a list or tuple"
            )

        for i, source in enumerate(sources):
            if not isinstance(source, str):
                raise TypeError(
                    f"{config_name}['csp']['{directive}'][{i}] must be a string"
                )


def _get_merged_config_for_request(app: Flask) -> Dict[str, Any]:
    """
    Get merged configuration for current request.

    Combines default configuration with route-specific overrides,
    performing deep merge for CSP directives.

    Args:
        app: Flask application instance

    Returns:
        Merged configuration dictionary for current request
    """
    config_data = app.extensions.get('advanced_secure_headers', {})
    default_config = config_data.get('default_config', {})
    route_config = config_data.get('route_config', {})

    # Start with deep copy of defaults
    merged = copy.deepcopy(default_config)

    # Find route-specific overrides
    endpoint = request.endpoint
    path = request.path

    # Check both endpoint name and path (path takes precedence)
    route_override = {}
    if endpoint and endpoint in route_config:
        route_override = route_config[endpoint]
    if path in route_config:
        route_override = route_config[path]

    # Merge overrides
    if route_override:
        merged = _deep_merge_configs(merged, route_override)

    return merged


def _deep_merge_configs(
    base_config: Dict[str, Any],
    override_config: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Deep merge two configuration dictionaries.

    Special handling for CSP: merges at directive level rather than
    replacing entire CSP dictionary.

    Args:
        base_config: Base configuration dictionary
        override_config: Override configuration dictionary

    Returns:
        New dictionary with merged configuration
    """
    result = copy.deepcopy(base_config)

    for key, override_value in override_config.items():
        if key == 'csp' and isinstance(override_value, dict):
            # Deep merge CSP directives
            if 'csp' not in result:
                result['csp'] = {}
            elif result['csp'] is None:
                result['csp'] = {}

            # Merge each CSP directive
            base_csp = result.get('csp', {}) or {}
            merged_csp = copy.deepcopy(base_csp)
            merged_csp.update(override_value)
            result['csp'] = merged_csp
        else:
            # Regular override for non-CSP keys
            result[key] = override_value

    return result


def _csp_requires_nonce(csp_config: Dict[str, Any]) -> bool:
    """
    Check if CSP configuration requires nonce generation.

    Args:
        csp_config: CSP configuration dictionary

    Returns:
        True if any directive contains nonce placeholder
    """
    if not isinstance(csp_config, dict):
        return False

    for directive_sources in csp_config.values():
        if isinstance(directive_sources, (list, tuple)):
            for source in directive_sources:
                if isinstance(source, str) and '{nonce}' in source:
                    return True

    return False


def _build_csp_header(
    csp_config: Dict[str, List[str]],
    nonce: Optional[str]
) -> str:
    """
    Build CSP header string from configuration dictionary.

    Args:
        csp_config: Dictionary mapping CSP directives to source lists
        nonce: Current request nonce for substitution

    Returns:
        Complete CSP header string

    Example:
        >>> config = {'default-src': ["'self'"], 'script-src': ["'nonce-{nonce}'"]}
        >>> _build_csp_header(config, 'abc123')
        "default-src 'self'; script-src 'nonce-abc123'"
    """
    if not csp_config:
        return ""

    directive_strings = []

    for directive, sources in csp_config.items():
        if not sources:
            continue

        # Process each source, substituting nonce if needed
        processed_sources = []
        for source in sources:
            if nonce and '{nonce}' in source:
                processed_source = source.replace('{nonce}', nonce)
            else:
                processed_source = source
            processed_sources.append(processed_source)

        # Join sources for this directive
        sources_str = ' '.join(processed_sources)
        directive_strings.append(f"{directive} {sources_str}")

    return '; '.join(directive_strings)


In [ ]:
# tests
import unittest
from flask import Flask, Response, render_template_string, jsonify
from main import apply_advanced_secure_headers, get_current_nonce


class TestAdvancedSecureHeaders(unittest.TestCase):

    def _create_app(self, default_cfg=None, route_cfg=None):
        app = Flask(__name__)
        apply_advanced_secure_headers(app, default_cfg or {}, route_cfg or {})
        return app

    @staticmethod
    def _get_header(resp: Response, key: str) -> str | None:
        for k, v in resp.headers.items():
            if k.lower() == key.lower():
                return v
        return None

    def test_default_headers_applied(self):
        default_cfg = {
            'csp': {'default-src': ["'self'"]},
            'hsts': 86400,
            'x_content_type_options': True,
        }
        app = self._create_app(default_cfg)

        @app.route('/')
        def index():
            return 'ok'

        with app.test_client() as client:
            resp = client.get('/')
            self.assertEqual(self._get_header(
                resp, 'Strict-Transport-Security'), 'max-age=86400')
            self.assertIn("default-src 'self'",
                          self._get_header(resp, 'Content-Security-Policy'))
            self.assertEqual(self._get_header(
                resp, 'X-Content-Type-Options'), 'nosniff')

    def test_route_specific_override(self):
        default_cfg = {
            'csp': {'default-src': ["'self'"]},
            'hsts': 86400,
        }
        route_cfg = {
            '/public': {
                'csp': {'default-src': ["'self'", 'cdn.example.com']},
                'hsts': 0,
            }
        }
        app = self._create_app(default_cfg, route_cfg)

        @app.route('/public')
        def public():
            return 'pub'

        with app.test_client() as client:
            resp = client.get('/public')
            # When hsts == 0 the reference impl omits the header entirely.
            self.assertIsNone(self._get_header(
                resp, 'Strict-Transport-Security'))
            csp = self._get_header(resp, 'Content-Security-Policy')
            self.assertIn('cdn.example.com', csp)

    def test_nonce_available_and_in_header(self):
        default_cfg = {
            'csp': {'script-src': ["'self'", "'nonce-{nonce}'"]},
        }
        app = self._create_app(default_cfg)

        @app.route('/dash')
        def dash():
            nonce = get_current_nonce()
            return render_template_string("<html>{{ nonce }}</html>", nonce=nonce)

        with app.test_client() as client:
            first = client.get('/dash')
            nonce_1 = first.get_data(as_text=True)[6:-7]
            self.assertIn(nonce_1, self._get_header(
                first, 'Content-Security-Policy'))
            second = client.get('/dash')
            nonce_2 = second.get_data(as_text=True)[6:-7]
            self.assertNotEqual(nonce_1, nonce_2)

    def test_non_html_response_headers(self):
        app = self._create_app({'csp': {'default-src': ["'none'"]}})

        @app.route('/json')
        def jsn():
            return jsonify(ok=True)

        with app.test_client() as client:
            resp = client.get('/json')
            self.assertIsNotNone(self._get_header(
                resp, 'Content-Security-Policy'))

    def test_disable_csp_and_hsts(self):
        default_cfg = {'csp': {'default-src': ["'self'"]}, 'hsts': 86400}
        route_cfg = {'/nocsp': {'csp': None, 'hsts': 0}}
        app = self._create_app(default_cfg, route_cfg)

        @app.route('/nocsp')
        def nocsp():
            return 'n'

        with app.test_client() as client:
            resp = client.get('/nocsp')
            self.assertIsNone(self._get_header(
                resp, 'Content-Security-Policy'))
            self.assertIsNone(self._get_header(
                resp, 'Strict-Transport-Security'))

    def test_route_added_after_apply(self):
        app = Flask(__name__)
        apply_advanced_secure_headers(
            app, {'csp': {'default-src': ["'self'"]}})

        @app.route('/later')
        def later():
            return 'ok'

        with app.test_client() as client:
            resp = client.get('/later')
            self.assertIn("default-src 'self'",
                          self._get_header(resp, 'Content-Security-Policy'))

    def test_unknown_route_inherits_defaults(self):
        default_cfg = {'csp': {'default-src': ["'self'"]}}
        route_cfg = {'/special': {'csp': {'default-src': ["'none'"]}}}
        app = self._create_app(default_cfg, route_cfg)

        @app.route('/other')
        def other():
            return 'x'

        with app.test_client() as client:
            resp = client.get('/other')
            self.assertIn("'self'", self._get_header(
                resp, 'Content-Security-Policy'))

    def test_invalid_config_type_error(self):
        app = Flask(__name__)
        bad_default = {'csp': 'not-a-dict'}
        with self.assertRaises(TypeError):
            apply_advanced_secure_headers(app, bad_default)

    def test_idempotent_after_request_handlers(self):
        """Re-applying should not produce duplicate CSP headers in response."""
        app = Flask(__name__)
        apply_advanced_secure_headers(
            app, {'csp': {'default-src': ["'self'"]}})
        apply_advanced_secure_headers(
            app, {'csp': {'default-src': ["'self'"]}})  # second call

        @app.route('/dup')
        def dup():
            return 'ok'

        with app.test_client() as client:
            resp = client.get('/dup')
            self.assertEqual(
                len(resp.headers.getlist('Content-Security-Policy')), 1)

    def test_blueprint_level_override(self):
        """Blueprint‑scoped config overrides defaults for its routes."""
        from flask import Blueprint
        default_cfg = {'csp': {'default-src': ["'self'"]}, 'hsts': 0}
        bp_cfg = {'bp.index': {
            'csp': {'default-src': ["'self'", 'static.example']}}}
        app = self._create_app(default_cfg, bp_cfg)
        bp = Blueprint('bp', __name__)

        @bp.route('/')
        def index():
            return 'bp'

        @app.route('/plain')
        def plain():
            return 'p'

        app.register_blueprint(bp, url_prefix='/bp')

        with app.test_client() as client:
            resp = client.get('/bp/')
            csp = self._get_header(resp, 'Content-Security-Policy')
            self.assertIn('static.example', csp)
            resp2 = client.get('/plain')
            self.assertNotIn('static.example', self._get_header(
                resp2, 'Content-Security-Policy'))


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/2f6aa67b-b7e2-4253-afdd-caf023318ab9?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F81781%2Fview

#### Model code:

```python
import secrets
import copy
from flask import g, request, current_app, Flask, render_template
from flask_talisman import Talisman


def get_current_nonce() -> str:
    """
    Return the cryptographically secure nonce string (e.g. "nonce-xxxx...") for the
    current request, or None if no nonce was set (non-HTML routes etc.).
    """
    return getattr(g, 'csp_nonce_value', None) # <Issue>: The nonce is generated and available programmatically, but it is never injected into the Flask/Jinja template environment. Template users must pass it explicitly through view functions, which is less ergonomic and not as robust as automatic context injection.


def apply_advanced_secure_headers(app, default_config=None, route_config=None):
    """
    Applies secure HTTP headers to the Flask 'app', combining global default
    configuration with per-route overrides.  Uses Flask-Talisman for the
    underlying mechanics but extends it with a custom after_request to handle
    route-specific policies and dynamic nonce injection.
    
    :param app: The Flask application instance.
    :param default_config: Optional dict of global defaults (e.g.):
        {
            'csp': {'default-src': ["'self'"]},
            'hsts': 86400,
            'x_content_type_options': True
        }
    :param route_config: Optional dict mapping path patterns or endpoint names
                         to per-route dicts. Same structure as default_config,
                         but overrides for just that route.
                         e.g. {
                           '/public': {
                               'csp': {'default-src': ["'self'", 'cdnjs.cloudflare.com']},
                               'hsts': 0
                           }
                         }
    """
    if default_config is None:
        default_config = {}
    if route_config is None:
        route_config = {}
    # <Issue>: There are no type, structure, or boundary checks for default_config and route_config; invalid values (e.g., string for hsts, or int for csp) will go unnoticed and may break the application or result in insecure headers. Misconfiguration should cause early failure, not silent malfunctions.

    # Stash configs in app.config for easy access in request hooks
    app.config['_ADVSEC_DEFAULT_CONFIG'] = default_config
    app.config['_ADVSEC_ROUTE_CONFIG'] = route_config

    #
    # 1. Initialize a *base* Talisman object with the global defaults.
    #
    #    We will then further refine/override these settings per-request
    #    in our custom after_request hook.  Talisman’s built-in "nonce"
    #    injection can be disabled here; we’ll implement our own notion
    #    of “nonce-...” to allow explicit 'nonce-{nonce}' placeholders.
    #
    talisman = Talisman(
        app,
        force_https=(default_config.get('hsts', 0) > 0),  # If we have an HSTS > 0, we assume HTTPS
        strict_transport_security=bool(default_config.get('hsts', 0)),
        strict_transport_security_max_age=default_config.get('hsts', 0),
        content_security_policy=default_config.get('csp', {}),
        content_security_policy_nonce_in=[],
        # We disable Talisman’s own nonce injection here, since
        # we'll manage the nonce string ourselves.
    )

    #
    # 2. Generate a request-specific nonce in before_request.
    #
    @app.before_request
    def _secure_headers_before_request():
        """
        Generates and stores a cryptographically secure nonce for each request.
        This allows inline scripts/styles to safely include the correct 'nonce-xxx'
        attribute and will be used in the final CSP header.
        """
        # Example: "nonce-m5TFIfQj..."
        nonce_str = "nonce-" + secrets.token_urlsafe(18)
        g.csp_nonce_value = nonce_str

    #
    # 3. Merge route-specific overrides in after_request.
    #
    @app.after_request
    def _secure_headers_after_request(response):
        """
        Applies final per-route security configuration by merging the default config
        with any route-specific overrides. Replaces 'nonce-{nonce}' in script or style
        directives with the actual nonce generated in before_request.
        """
        # Identify the route or endpoint
        endpoint = request.endpoint
        path = request.path

        # Retrieve global defaults and (possible) per-route overrides
        global_cfg = app.config.get('_ADVSEC_DEFAULT_CONFIG', {})
        route_cfgs = app.config.get('_ADVSEC_ROUTE_CONFIG', {})

        # A route config can be keyed by path or by endpoint name.  Check both
        # in whichever order you prefer.  If neither is present, use only the global cfg.
        current_override = route_cfgs.get(path) or route_cfgs.get(endpoint) or {}

        # Merge the final configuration
        final_cfg = copy.deepcopy(global_cfg)
        for k, v in current_override.items():
            final_cfg[k] = v # <Issue>: This code replaces the entire 'csp' dictionary if an override exists, rather than merging individual directives. If a route-specific config contains only 'script-src', it wipes out global 'default-src' (or other global directives), violating the security intent.

        # (a) Strict-Transport-Security
        hsts_age = final_cfg.get('hsts', 0)
        if hsts_age > 0:
            response.headers['Strict-Transport-Security'] = f'max-age={hsts_age}'
        else:
            # Remove if override or route specifically sets hsts=0
            response.headers.pop('Strict-Transport-Security', None)

        # (b) X-Content-Type-Options
        if final_cfg.get('x_content_type_options', False):
            response.headers['X-Content-Type-Options'] = 'nosniff'
        else:
            response.headers.pop('X-Content-Type-Options', None)

        # (c) Content-Security-Policy
        csp_map = final_cfg.get('csp', {})
        csp_nonce = get_current_nonce()  # e.g. "nonce-XXXX..."
        csp_header_value = _build_csp_header_value(csp_map, csp_nonce)

        if csp_header_value:
            response.headers['Content-Security-Policy'] = csp_header_value
        else:
            # If no CSP is given for the route, remove the header (or leave it out)
            response.headers.pop('Content-Security-Policy', None)

        return response

    #
    # 4. Utility: build the actual CSP header from a dictionary of directives,
    #    substituting nonce placeholders if present.
    #
    def _build_csp_header_value(csp_directives, nonce): # <Issue>: Errors for invalid CSP structure only occur at response build time, rather than when applying the security configuration. This may cause silent startup with runtime failures later. Validation should occur at configuration time.
        """
        Convert a dict like {'default-src': ["'self'"], 'script-src': ["'self'", "'nonce-{nonce}'"]}
        into the CSP header string: "default-src 'self'; script-src 'self' 'nonce-...'"
        
        :param csp_directives: dict of directive -> list/string of sources
        :param nonce: the actual nonce value to substitute into any 'nonce-{nonce}' placeholders
        :return: The full CSP header string (or "" if empty).
        """
        if not csp_directives:
            return ""

        parts = []
        for directive, sources in csp_directives.items():
            if isinstance(sources, (list, tuple)):
                source_list = [
                    src.replace('{nonce}', nonce) if '{nonce}' in src and nonce else src
                    for src in sources
                ]
                parts.append(f"{directive} {' '.join(source_list)}")
            elif isinstance(sources, str):
                final_src = sources.replace('{nonce}', nonce) if '{nonce}' in sources and nonce else sources
                parts.append(f"{directive} {final_src}")
            else:
                # If it is neither list nor string, skip or raise an error
                raise TypeError(
                    f"CSP directive {directive} must be a string or list/tuple of strings."
                )

        return "; ".join(parts)

```